In [1]:
labels = {
    # Modalidad de trabajo
    "remote": [
        "The position allows remote work.",
        "This is a fully remote job.",
        "You can work from anywhere.",
    ],
    "permanent_contract": [
        "The job offers a permanent contract.",
        "This is a full-time position.",
    ],
    "freelance": [
        "This job is freelance.",
        "The position is for an independent contractor.",
    ],

    # Beneficios y condiciones
    "bonuses": [
        "The job offers performance bonuses.",
        "You will receive bonuses based on results.",
        "There are financial incentives in this position.",
    ],
    "career_development": [
        "There are opportunities for professional growth.",
        "The company supports career development.",
    ],
    "immediate_start": [
        "The job requires immediate start.",
        "The position must be filled urgently.",
        "The company is looking for someone to start right away.",
    ],

    # Tipo de empresa
    "startup": [
        "The company is a startup.",
        "You will be working in a fast-growing new company.",
        "The role is at an early-stage company.",
    ],
    "multinational": [
        "The company operates in multiple countries.",
        "This is a multinational organization.",
        "The employer has offices worldwide.",
    ],
    "consulting_or_outsourcing": [
        "The job is in a consulting company.",
        "You will work in an outsourcing services firm.",
        "The company provides external services to clients.",
    ],

    # Experiencia requerida
    "experience_gt_5": [
        "You need more than five years of experience.",
        "The job requires at least 5 years of work experience.",
        "Senior professionals with over 5 years of experience are needed.",
    ],

    # Nivel del puesto
    "intern": [
        "The position is for an intern.",
        "The role is aimed at students or recent graduates.",
    ],
    "junior": [
        "The position is at junior level.",
        "This role is for someone starting their career.",
    ],
    "senior": [
        "This is a senior position.",
        "The job requires significant experience.",
    ],
    "lead": [
        "You will lead a team.",
        "The position is for a technical or team lead.",
        "You are expected to guide other professionals.",
    ],
    "principal_or_manager": [
        "You will manage a department or team.",
        "This is a managerial role.",
        "You are responsible for strategic decisions.",
    ],

    # Educación y certificaciones
    "higher_education": [
        "A university degree is required.",
        "The role requires a bachelor's, master's, or PhD.",
    ],
    "certifications_required": [
        "Professional certifications are required.",
        "You need official credentials or certifications.",
    ],

    # Ámbitos profesionales
    "software_development": [
        "The job is in software development.",
        "You will be writing code or developing software.",
        "The role involves programming.",
    ],
    "data_science_ml": [
        "The position is in data science.",
        "Machine learning is a part of this job.",
        "You will analyze data or build ML models.",
    ],
    "cybersecurity": [
        "The job is in cybersecurity.",
        "The role involves protecting systems and data.",
        "You will handle security threats.",
    ],
    "qa_testing": [
        "You will be testing software for quality.",
        "The job involves quality assurance.",
        "You will perform tests to find bugs.",
    ],
    "it_support_infrastructure": [
        "The job involves IT support.",
        "You will maintain computer systems and networks.",
        "Infrastructure management is part of the role.",
    ],
    "project_product_management": [
        "The role involves managing projects or products.",
        "You will be a product or project manager.",
        "The position requires planning and coordination.",
    ],
    "ux_ui_design": [
        "The job is in UX or UI design.",
        "You will design user interfaces or experiences.",
        "The role focuses on user-centered design.",
    ]
}


In [2]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch
import statistics
import os

In [3]:
df = pd.read_csv('LIMPIO.csv')
df.shape

(120784, 13)

In [5]:
import os
import pandas as pd
import torch
from transformers import pipeline
from tqdm import tqdm
import statistics
import pickle

# Cargar dataset
df = pd.read_csv('LIMPIO.csv')

# Descripciones de trabajo
descriptions = df["description_full"].fillna('')

# Detectar si hay GPU disponible
device = 0 if torch.cuda.is_available() else -1

# Cargar modelo zero-shot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# Asegurar que el diccionario 'labels' está definido
label_dict = labels  # <- asegúrate de que esté definido antes de ejecutar

# Inicializar resultados
results = {key: [] for key in label_dict}

# Parámetros de guardado
chunk_size = 1000
output_file = 'MODELADO_FINAL.csv'
checkpoint_file = 'checkpoint.pkl'

# Cargar checkpoint si existe
start_index = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)
        results = checkpoint['results']
        start_index = checkpoint['last_index'] + 1
        print(f"🔁 Reanudando desde la fila {start_index}")

# Clasificar descripciones con barra de progreso
progress_bar = tqdm(range(start_index, len(descriptions)), desc="🔍 Procesando descripciones")

for i in progress_bar:
    desc = descriptions[i]
    try:
        for category, prompts in label_dict.items():
            scores = []
            for prompt in prompts:
                output = classifier(desc, candidate_labels=[prompt])
                scores.append(output['scores'][0])
            avg_score = statistics.mean(scores)
            results[category].append(avg_score)
    except Exception as e:
        print(f"\n⚠️ Error en desc {i}: {str(e)[:80]}...")
        for category in label_dict:
            results[category].append(0)

    # Guardado parcial y checkpoint
    if (i + 1) % chunk_size == 0 or (i + 1) == len(descriptions):
        partial_df = df.iloc[:i+1].copy()
        for category, scores in results.items():
            partial_df[category] = scores[:i+1]

        # Guardado del CSV
        if not os.path.isfile(output_file):
            partial_df.to_csv(output_file, index=False)
        else:
            partial_df.tail(chunk_size).to_csv(output_file, mode='a', header=False, index=False)

        print(f"\n✅ Guardado parcial en fila {i+1}")

        # Guardado del checkpoint
        checkpoint_data = {
            'results': results,
            'last_index': i
        }
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(checkpoint_data, f)
        print(f"💾 Checkpoint guardado en fila {i+1}")

# Guardado final completo
for category, scores in results.items():
    df[category] = scores

df.to_csv(output_file, index=False)

# Eliminar el checkpoint final
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print("🗑️ Checkpoint eliminado.")

print(f"✅ Zero-Shot completado y guardado en: {output_file}")


Device set to use cuda:0
🔍 Procesando descripciones:   1%|          | 1000/120784 [47:31<146:37:10,  4.41s/it]


✅ Guardado parcial en fila 1000
💾 Checkpoint guardado en fila 1000


🔍 Procesando descripciones:   2%|▏         | 2000/120784 [1:45:40<92:25:10,  2.80s/it] 


✅ Guardado parcial en fila 2000
💾 Checkpoint guardado en fila 2000


🔍 Procesando descripciones:   2%|▏         | 3000/120784 [2:46:15<100:12:22,  3.06s/it]

...

🔍 Procesando descripciones:  99%|█████████▉| 120000/120784 [125:56:38<35:21,  2.71s/it]  


✅ Guardado parcial en fila 120000
💾 Checkpoint guardado en fila 120000


🔍 Procesando descripciones: 100%|██████████| 120784/120784 [126:17:20<00:00,  3.76s/it]


✅ Guardado parcial en fila 120784
💾 Checkpoint guardado en fila 120784


🗑️ Checkpoint eliminado.
✅ Zero-Shot completado y guardado en: MODELADO_FINAL.csv


In [ ]:
# Recargar el DataFrame con clasificaciones, si fuera necesario
df = pd.read_csv('MODELADO_FINAL.csv')

# Agregar columnas de roles según el título
df["intern_title"] = df["title"].str.contains(r"\bintern|trainee\b", case=False, na=False).astype(int)
df["junior_title"] = df["title"].str.contains(r"\bjunior|jr\b", case=False, na=False).astype(int)
df["senior_title"] = df["title"].str.contains(r"\bsenior|sr\b", case=False, na=False).astype(int)
df["lead_title"] = df["title"].str.contains(r"\blead|team lead|tech lead\b", case=False, na=False).astype(int)
df["principal_or_manager_title"] = df["title"].str.contains(r"\bprincipal|manager|head\b", case=False, na=False).astype(int)

# Guardar con roles añadidos
df.to_csv('MODELADO_FINAL_CON_TITULOS.csv', index=False)

print("✅ Columnas de roles añadidas y archivo actualizado: MODELADO_FINAL_CON_TITULOS.csv")